<a href="https://colab.research.google.com/github/IvanovMaxim2000/diploma_MAI_2024/blob/main/bert_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
pip install accelerate -U

In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00


In [4]:
pip install torch

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [6]:
import pandas as pd
import numpy as np
import re
import os
import torch
import pickle
import sklearn
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoModel,AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer
from nltk.corpus import stopwords
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from nltk.stem.snowball import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize, word_tokenize
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoModel, BertModel, AutoTokenizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, precision_recall_fscore_support

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
data = load_dataset('csv', data_files={'train': 'df_file.csv'})

# Преобразование меток в числовой формат (если необходимо)
data = data['train'].map(lambda example: {'label': int(example['Label'])})

# Выбор и загрузка модели
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Токенизация
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True, return_tensors="pt")

data = data.map(tokenize_function, batched=True)

# Разделение на обучающую и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2)

train_data = Dataset.from_dict(train_data)
test_data = Dataset.from_dict(test_data)

# Обучение модели
training_args = TrainingArguments(output_dir="./results", evaluation_strategy="epoch")
trainer = Trainer(model=model, args=training_args, train_dataset=train_data, eval_dataset=test_data)

# Проверка данных перед обучением
print(train_data[0])

# Запуск обучения
trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Map:   0%|          | 0/2225 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


{'Text': 'More power to the people says HP\n \n The digital revolution is focused on letting people tell and share their own stories, according to Carly Fiorina, chief of technology giant Hewlett Packard.\n \n The job of firms such as HP now, she said in a speech at the Consumer Electronics Show (CES), was to ensure digital and physical worlds fully converged. She said the goal for 2005 was to make people the centre of technology. CES showcases 50,000 new gadgets that will be hitting the shelves in 2005. The tech-fest, the largest of its kind in the world, runs from 6 to 9 January. "The digital revolution is about the democratisation of technology and the experiences it makes possible," she told delegates. "Revolution has always been about giving power to the people." She added: "The real story of the digital revolution is not just new products, but the millions of experiences made possible and stories that millions can tell." Part of giving people more control has been about the freei

Epoch,Training Loss,Validation Loss
1,No log,0.063427
2,No log,0.083782
3,0.152600,0.097404


TrainOutput(global_step=669, training_loss=0.11706749313080791, metrics={'train_runtime': 33056.8141, 'train_samples_per_second': 0.162, 'train_steps_per_second': 0.02, 'total_flos': 1405050880757760.0, 'train_loss': 0.11706749313080791, 'epoch': 3.0})

In [8]:
# получение предсказаний модели
predictions = trainer.predict(test_data)
predicted_labels = predictions.predictions.argmax(-1)
true_labels = test_data['Label']

accuracy = accuracy_score(true_labels, predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# 5. Сохранение модели
trainer.save_model("bert_res")

Accuracy: 0.9865168539325843
Precision: 0.9867786625940875
Recall: 0.9865168539325843
F1-score: 0.9865310866694604
